In [4]:
import qrcode
import sqlite3
from flask import Flask, send_file, request, jsonify
import io
import uuid

# Initialize the Flask app
app = Flask(__name__)

# In-memory database to store users who have scanned the code
conn = sqlite3.connect(':memory:', check_same_thread=False)
cursor = conn.cursor()

# Create a table to track users
cursor.execute('''CREATE TABLE users (user_id TEXT PRIMARY KEY, scanned INTEGER)''')
conn.commit()

# Generate a QR code
def generate_qr(data):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)
    
    img = qr.make_image(fill='black', back_color='white')
    return img

# Route to generate and serve a one-time QR code
@app.route('/generate_qr')
def get_qr():
    # Generate a unique ID for the user/session
    user_id = str(uuid.uuid4())
    
    # Store the user in the database (set scanned = 0 initially)
    cursor.execute('INSERT INTO users (user_id, scanned) VALUES (?, ?)', (user_id, 0))
    conn.commit()
    
    # Create the QR code with the unique user ID embedded
    qr_img = generate_qr(user_id)
    
    # Convert the image to a byte stream to send as a file response
    img_io = io.BytesIO()
    qr_img.save(img_io, 'PNG')
    img_io.seek(0)
    
    return send_file(img_io, mimetype='image/png')

# Route to handle the QR code scan
@app.route('/scan_qr/<user_id>', methods=['POST'])
def scan_qr(user_id):
    # Check if the user exists and if they have already scanned the QR code
    cursor.execute('SELECT scanned FROM users WHERE user_id = ?', (user_id,))
    result = cursor.fetchone()
    
    if result is None:
        return jsonify({'error': 'Invalid QR code'}), 404
    elif result[0] == 1:
        return jsonify({'error': 'QR code has already been used'}), 400
    else:
        # Mark the QR code as used
        cursor.execute('UPDATE users SET scanned = 1 WHERE user_id = ?', (user_id,))
        conn.commit()
        return jsonify({'success': 'QR code scanned successfully'}), 200

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True,port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1